In [1]:
import hoomd
import hoomd.hpmc
import coxeter
from ipywidgets import interact, interactive
import ipywidgets as widgets
import plato, plato.draw.vispy as draw
import gsd, gsd.hoomd
%pylab inline

Populating the interactive namespace from numpy and matplotlib


/Users/mayamartirossyan/anaconda3/envs/closet/lib/python3.8/site-packages/IPython/core/magics/pylab.py:159: UserWarning: pylab import has clobbered these variables: ['interactive', 'draw']
`%matplotlib` prevents importing * from pylab and numpy
  warn("pylab import has clobbered these variables: %s"  % clobbered +


In [ ]:
#coxeter shapes RegularNGON family in 2D -- Rachael

In [2]:
hoomd.context.initialize("--mode=cpu");

HOOMD-blue 2.9.7 DOUBLE HPMC_MIXED TBB SSE SSE2 SSE3 
Compiled: 08/09/2021
Copyright (c) 2009-2019 The Regents of the University of Michigan.
-----
You are using HOOMD-blue. Please cite the following:
* J A Anderson, J Glaser, and S C Glotzer. "HOOMD-blue: A Python package for
  high-performance molecular dynamics and hard particle Monte Carlo
  simulations", Computational Materials Science 173 (2020) 109363
-----
-----
You are using HPMC. Please cite the following:
* J A Anderson, M E Irrgang, and S C Glotzer. "Scalable Metropolis Monte Carlo
  for simulation of hard shapes", Computer Physics Communications 204 (2016) 21
  --30
-----
HOOMD-blue is running on the CPU


In [22]:
# We can use text input
shape_1 = 'square'

In [26]:
# We can also use more interactive input methods
shape_2 = widgets.Text(
    value='',
    placeholder='Type something',
    description='2nd shape:',
    disabled=False
)

display(shape_2)

Text(value='', description='2nd shape:', placeholder='Type something')

In [19]:
print(shape_2.value) #this is how you access the string for the second shape chosen

In [32]:
# Choose a fraction between 0 and 1
# which will represent the percent of shapes
# that are Shape 1
# shape_ratio = 0 means all the shapes will be Shape 2

shape_ratio = widgets.FloatSlider(
    min=0.00, 
    max=1.00, 
    step=0.01, 
    description='Shape ratio:', 
    value=0.50,
    readout_format='.2f')

display(shape_ratio)

FloatSlider(value=0.5, description='Shape ratio:', max=1.0, step=0.01)

In [34]:
print(shape_ratio.value)

0.81


In [ ]:
numParticles_sqrt = 10 # example: if numParticles_sqrt = 10, then there are 100 particles in the system

In [ ]:
system = hoomd.init.create_lattice(unitcell=hoomd.lattice.sq(a=3),
                                   n=numParticles_sqrt);

In [ ]:
#actual simulation setup goes here

In [ ]:
#graph volume over time -- Rachael
#graph packing fraction over time -- Rachael

### Plato visualizations

In [ ]:
def getFrameCount(fname):
    #inputs: fname, the filename (ex: 'dump.gsd')
    #outputs: len(traj), number of frames in simulation
    with gsd.hoomd.open(fname, 'rb') as traj:
        return len(traj)

In [ ]:
prim = draw.Polygons() #how to get it to consider different polygons?
box_prim = draw.Box(color=(0, 0, 0, 1), width=.2)
scene = draw.Scene([prim, box_prim], zoom=.5, clip_scale=5)
scene.show()

filename = 'dump.gsd'
frame_num = getFrameCount(filename)

@ipywidgets.interact(frame_index=range(frame_num))
def plot(frame_index=0):
    with gsd.hoomd.open(filename, 'rb') as traj:
        frame = traj[frame_index]
        box = frame.configuration.box
        for (name, val) in zip(['Lx', 'Ly', 'Lz', 'xy', 'xz', 'yz'], box):
            setattr(box_prim, name, val)
        prim.positions = frame.particles.position
        prim.diameters = np.full(len(frame.particles.position), 1)
    scene.render()